In [10]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

#reading in the data file
wellData = pd.read_csv("Data/well production.csv")
wellData.head()

,well name,average pressure (Pa),recovery factor,formation volume factor,oil 1,oil 2,oil 3,oil 4,oil 5,oil 6,...,water 3,water 4,water 5,water 6,water 7,water 8,water 9,water 10,water 11,water 12
0,Peak 6-217H,35352874,0.092554,1.6,862.0,824.0,759.0,728.0,661.0,601.0,...,180.0,211.0,277.0,338.0,397.0,445.0,547.0,562.0,639.0,668.0
1,Tarragon 4-119H,34882173,0.107706,1.6,228.0,249.0,214.0,210.0,161.0,163.0,...,22.0,27.0,75.0,74.0,59.0,90.0,124.0,119.0,126.0,157.0
2,Fennel 10-129H,36064538,0.079150,1.6,67.0,85.0,73.0,73.0,57.0,58.0,...,15.0,15.0,31.0,30.0,33.0,31.0,20.0,49.0,30.0,41.0
3,Federal 14-113H,35817881,0.103748,1.6,256.0,242.0,267.0,263.0,199.0,191.0,...,9.0,13.0,78.0,86.0,119.0,134.0,139.0,162.0,136.0,183.0
4,King 7-184H,38442406,0.084675,1.6,23.0,29.0,31.0,50.0,72.0,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0


In [11]:
# general dataframe with information about all wells
genData = pd.DataFrame(columns = ['name', 'length', 'stageCount', 'propWeight', 
                                  'pumpRate', 'easting', 'northing', 'oilProd'])

# for every well logged
for i in range(len(wellData.index)):
    name = wellData.iloc[i,0]
    # creating file name to be read
    fileName = f"Data/{name}.csv"
    tempDF = pd.read_csv(fileName)
    # storing csv of current well
    tempDF.columns = ['east', 'north', 'por', 'perm', 'poisson', 'young', 'waterSat', 
                      'oilSat', 'thick', 'propWeight', 'pumpRate']
    # finds total well length
    wellLen = tempDF.iloc[len(tempDF.index) - 1, 0] - tempDF.iloc[0, 0]
    # number of stages, used for averages
    stageCount = tempDF['propWeight'].count()
    # calculates average proppant weight per stage
    avgProp = tempDF['propWeight'].sum() / stageCount
    # calculates average pump rate for stage
    avgPumpRate = tempDF['pumpRate'].sum() / stageCount
    # calculates total yearly well production
    prod = wellData.iloc[i, 4 : 16].sum()
    #calculate locations of wells 
    easting = tempDF['east'].mean()
    northing = tempDF['north'].mean()
    #appends well-specific information into general dataframe for plotting
    genData = genData.append(pd.Series([name, wellLen, stageCount, avgProp, avgPumpRate,
                                       easting, northing, prod], 
                             index = genData.columns), ignore_index = True)

In [12]:
def scatter(df, xCol, yCol):
    x = df[xCol]
    y = df[yCol]
    
    fig, ax = plt.subplots(figsize = (10, 5))
    ax.scatter(x, y)
    ax.set_title(f"{yCol} vs {xCol}")
    
    ax.set_xlabel(xCol)
    ax.set_ylabel(yCol)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.show()
    
def pow(list, power):
    return [x**power for x in list]



    

In [15]:
for name in range(len(genData.columns) - 1):
    scatter(genData, genData.columns[i], len(genData.columns))

KeyError: 99

In [14]:
genData.head()

,name,length,stageCount,propWeight,pumpRate,easting,northing,oilProd
0,Peak 6-217H,7565.0,100,910422.941447,294.800448,81782.32,1600.0,6808.0
1,Tarragon 4-119H,9643.0,50,728628.013775,288.362177,93221.44,32200.0,1968.0
2,Fennel 10-129H,10963.0,25,988860.773545,290.685170,13681.29,200.0,736.0
3,Federal 14-113H,10664.0,34,690947.011162,298.807597,84831.86,31000.0,2204.0
4,King 7-184H,7276.0,50,832067.579729,295.943915,9038.00,79200.0,490.0


In [21]:
noName = genData[['length', 'stageCount', 'propWeight', 'pumpRate', 'easting', 'northing']]
y = genData['oilProd'].values
x_train, x_test, y_train, y_test = train_test_split(noName, y, test_size = .1)

n_estimators = [int(x) for x in np.linspace(start=100, stop=900, num=100)]
# the number of features to use at each split
max_features = ["auto", "sqrt"]
# max number of levels in each tree
max_depth = [int(x) for x in np.linspace(10, 220, num=11)]
max_depth.append(None)
# minimum samples needed to split a tree
min_samples_split = [2, 5, 10, 15, 20]
# minimum samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8, 16]
# method for selecting samples
bootstrap = [True, False]


# create the grid
random_grid = {
    "n_estimators": n_estimators,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "bootstrap": bootstrap,
}

7706.73


In [22]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=random_grid,
    n_iter=100,
    cv=3,
    verbose=2,
    random_state=19,
    n_jobs=-1,
)
rf_random.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   11.4s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [23]:
def compare(model, test_features, test_actual):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_actual)
    mape = 100 * np.mean(errors / test_actual)
    accuracy = 100 - mape
    print("Model Performance")
    print("Average Error: {:0.4f}".format(np.mean(errors)))
    print("Accuracy = {:0.2f}%.".format(accuracy))

    return accuracy

base_model = RandomForestRegressor(n_estimators=10)
base_model.fit(x_train, y_train)
base_accuracy = compare(base_model, x_train, y_train)

best_random = rf_random.best_estimator_
random_accuracy = compare(best_random, x_train, y_train)

Model Performance
Average Error: 182.1344
Accuracy = 87.35%.
Model Performance
Average Error: 0.0000
Accuracy = 100.00%.


In [24]:
feat_labels = noName.columns.values  # get the feature labels
feature = list(
    zip(feat_labels, best_random.feature_importances_)
)  # make a list of the feature labels and the importance values
sorted(
    feature, key=lambda tup: tup[1], reverse=True
)  # sort from most to least important feature in predicting production

[('stageCount', 0.3870416576540284),
 ('northing', 0.21586886405660746),
 ('easting', 0.19581377356003252),
 ('propWeight', 0.0723931527932072),
 ('pumpRate', 0.066944151307439),
 ('length', 0.0619384006286853)]

In [233]:
from sklearn.model_selection import cross_val_score, GridSearchCV

new = noName.drop(columns = ['propWeight', 'pumpRate', 'length'])
x_train, x_test, y_train, y_test = train_test_split(new, y, test_size = .1) 
newRf = RandomForestRegressor(n_estimators = 100)
newRf.fit(x_train, y_train)
scores = cross_val_score(newRf, x_test, y_test, cv=10, scoring='neg_mean_absolute_error')

In [239]:
easting_list = [int(x) for x in np.linspace(10000, 100000, num = 10)]
northing_list = [int(x) for x in np.linspace(5000, 100000, num = 20)]
stage_count_list = [int(x) for x in np.linspace(4, 100, num = 25)]
tuple_list = []
for i in range(len(easting_list)):
    for j in range(len(northing_list)):
        east = easting_list[i] - 5000
        north = northing_list[j] - 2500
        max_stage = 0
        max_val = 0
        for k in range(len(stage_count_list)):
            stage = stage_count_list[k] - 2
            curr_pred = newRf.predict([[stage, east, north]])
            if curr_pred > max_val:
                max_val = curr_pred
                max_stage = stage
        tuple_list.append(((stage, east, north), max_val))

In [238]:
def getKey(item):
    return item[1]
sorted_tuple_list = sorted(tuple_list, key = getKey, reverse = True)
sorted_tuple_list

[((98, 65000, 17500), array([4548.2])),
 ((98, 75000, 17500), array([4519.7])),
 ((98, 65000, 12500), array([4440.57])),
 ((98, 75000, 12500), array([4413.87])),
 ((98, 65000, 22500), array([4314.09])),
 ((98, 75000, 22500), array([4270.51])),
 ((98, 65000, 27500), array([4265.14])),
 ((98, 75000, 7500), array([4217.17])),
 ((98, 65000, 7500), array([4205.75])),
 ((98, 75000, 27500), array([4192.63])),
 ((98, 75000, 2500), array([4187.3])),
 ((98, 65000, 2500), array([4179.97])),
 ((98, 85000, 17500), array([4118.44])),
 ((98, 85000, 12500), array([4014.29])),
 ((98, 85000, 22500), array([3968.94])),
 ((98, 85000, 7500), array([3884.64])),
 ((98, 85000, 27500), array([3880.31])),
 ((98, 85000, 2500), array([3876.94])),
 ((98, 55000, 17500), array([3788.76])),
 ((98, 55000, 12500), array([3722.81])),
 ((98, 95000, 17500), array([3713.14])),
 ((98, 65000, 32500), array([3660.88])),
 ((98, 55000, 22500), array([3653.45])),
 ((98, 55000, 27500), array([3627.19])),
 ((98, 95000, 12500), arr

In [232]:
newlist = [(sorted_tuple_list[i][0][1], sorted_tuple_list[i][0][2]) for i in range(10)]
print(newlist)

[(65000, 17500), (65000, 12500), (75000, 17500), (75000, 12500), (65000, 22500), (65000, 7500), (65000, 27500), (65000, 2500), (75000, 7500), (75000, 22500)]


In [12]:
import pandas as pd
hi = []
hi.append(5)
hi.append(6)
yum = pd.Series(hi, index = ['hi', 'hi'])
yum

hi    5
hi    6
dtype: int64

In [16]:
import numpy as np

list_hi = np.array([1, 2, 3, 4, 5])
print(list_hi)

[1 2 3 4 5]
